In [67]:
from selenium import webdriver
import pandas as pd

from selenium.common.exceptions import NoSuchElementException

import time

In [135]:
driver = webdriver.Chrome()

In [3]:
tsherpa_map_search_url = "https://edu.tsherpa.co.kr/LectureInfo/LectureReviews2"

In [136]:
driver.get(tsherpa_map_search_url)

In [ ]:
# 연도: 2023, 2022까지만 수집 
# 수집 후 데이터프레임 생성 후 csv 파일로 저장 

In [137]:
ind = 1  # 현재 복사한 순서 # 인덱스
no = 1  # 1~10페이지 중 위치한 곳
page = 1  # 현재 페이지 번호
list2023_22 = []  # 결과물이 저장되는 리스트

while 1: # 무한반복
    try:
        try:
            img_element = driver.find_element(by='xpath', value=f'//*[@id="mainForm"]/table/tbody/tr[{ind}]/td[2]/a/img')
            # 이미지가 있는 경우 데이터 수집
            if img_element.is_displayed():
                
                date_ = driver.find_element(by='xpath', value=f'//*[@id="mainForm"]/table/tbody/tr[{ind}]/td[3]').text
                date = date_.split('\n')[1] # 등록일
                
                date_year = date.split('.')[0] # 연도만 추출 
                
                if date_year == '2021':
                    break
                else:
                    # 플러스 이미지 클릭 
                    driver.find_element(by='xpath', value=f'//*[@id="mainForm"]/table/tbody/tr[{ind}]/td[2]/a/img').click()
                    # 열린 새로운 창으로 이동 
                    driver.switch_to.window(driver.window_handles[-1])
                    time.sleep(1) # 크롤링할 시간을 줌 

                    title = driver.find_element(by='xpath', value=f'//*[@id="mainForm"]/div[1]/span/a').text  # 연수명
                    content = driver.find_element(by='xpath', value=f'//*[@id="mainForm"]/div[2]/table/tbody/tr[3]/td').text  # 내용
                    img_element = driver.find_element(by='xpath', value=f'//*[@id="mainForm"]/div[2]/table/tbody/tr[2]/td/span[1]/img')
                    score = img_element.get_attribute('alt')

                    driver.close() # 창 닫음 
                    driver.switch_to.window(driver.window_handles[0]) # 다시 현재의 창으로 이동 

                    # 수집한 데이터를 리스트에 넣음 
                    list2023_22.append([title, content, score, date])    

        except NoSuchElementException:
            # 이미지가 없는 경우 데이터 수집
            
                date_ = driver.find_element(by='xpath', value=f'//*[@id="mainForm"]/table/tbody/tr[{ind}]/td[3]').text
                date = date_.split('\n')[1] # 등록일 
                date_year = date.split('.')[0] # 연도만 추출 
                
                if date_year == '2021':
                    break
                else:
                    review = driver.find_element(by='xpath', value=f'//*[@id="mainForm"]/table/tbody/tr[{ind}]/td[2]/a').text
                    title = review.split('\n\n')[0]  # 연수명
                    content = review.split('\n\n')[1].split('\n')[1]  # 내용
                    img_element = driver.find_element(by='xpath', value=f'//*[@id="mainForm"]/table/tbody/tr[{ind}]/td[3]/img')
                    score = img_element.get_attribute('alt') # 점수

                    list2023_22.append([title, content, score, date])

        finally:
            # 다음 인덱스로 이동
            ind += 1
        
        # 쭉 가져오다가. 10페이지를 다 돌았으면, 다음페이지를 누르고 아니면 계속 페이지 이동을 해라 
    except: 
        # 위치를(no.) 10번을 다 돌았는지 확인. 
        # 다 돌았으면 다음 페이지 버튼을 클릭하고 페이지번호와 인덱스를 다시 1로 바꾸고 페이지 번호는 하나 늘림
        # 10페이지 단위마다 다음 페이지 버튼 누르기
        if no >= 10:
            driver.find_element(by='xpath',value = f'//*[@id="mainForm"]/div[3]/a[12]/img').click()
            no = 1
            ind = 1
            page += 1
            continue
            
        # 위치를 다 돌지않았으면 페이지 번호를 클릭함 
        elif driver.find_element(by='xpath',value = f'//*[@id="mainForm"]/div[3]/a[{no+2}]').is_displayed():
            no += 1
            driver.find_element(by='xpath',value = f'//*[@id="mainForm"]/div[3]/a[{no+1}]').click()
            ind = 1
            page += 1
            continue
        
        # 더이상 이동할수 없을 경우 종료
        # = 다음페이지도 없고 10배수도 아닐때 종료 
        else:
            break

In [141]:
list2023_22 = pd.DataFrame(list2023_22)

In [143]:
list2023_22

,0,1,2,3
0,같이 읽고 함께 나누는 독서수업방법,아이를 키우며 그림책과 동화책에 관심을 갖게 되었다.\n감동적이고 재미있는 책들을 ...,5점,2023.07.28
1,강의의 품격,비추천합니다. 새로울 것 없는 내용과 화질 안 좋은 것은 참겠는데 음향은 정말 괴로...,3점,2023.07.27
2,신병주 교수의 왕으로 산다는 것,조선왕들의 역사를 통하여 현재를 묻는 거울이 된다.\n균형있는 강의 내용과 치우침이...,5점,2023.07.27
3,15시간 메디컬 콘서트 1탄 - 현대인의 만성 질병,잘못 알고 있던 의학 상식들을 제대로 알 수 있어 좋았어요,5점,2023.07.27
4,정훈쌤의 뚝딱 칼림바,칼림바 연수에서 칼림바와 악보를 선물로 받았습니다. 학생들이 음악 강사와 칼림바 수...,5점,2023.07.26
...,...,...,...,...
1405,6시간 한 번에 끝내는 법정연수 1,감사합니다.,5점,2022.01.04
1406,30시간 그림검사와 다양한 기법으로 만나는 미술치료,강사분이 적극적으로 자세히 설명하는 부분이 너무 좋음,5점,2022.01.03
1407,6시간 한 번에 끝내는 법정연수 1,유익한 연수였습니다. 잘 들었습니다.,5점,2022.01.03
1408,당장 따라 할 수 있는 사계절 학급운영+이벤트,담임 경력이 짧은데 내년 학년 아직 결정도 안 된 상태에서 내년 걱정만으로 겨울방학...,5점,2022.01.03


In [142]:
list2023_22.to_csv('t_data_2.csv')